In [1]:
import os
from dotenv import load_dotenv

load_dotenv("../../_apikeys.env")
api_key = os.getenv("DoogieOpenaiKey")
os.environ['OPENAI_API_KEY'] = api_key

In [2]:
import numpy as np
from numpy import dot
from numpy.linalg import norm

from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import PromptTemplate
from langchain_core.tracers import ConsoleCallbackHandler
from langchain_community.chat_message_histories import ChatMessageHistory

In [2]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
#from langchain_core.runnables.history import RunnableWithMessageHistory
#from langchain_community.chat_message_histories import ChatMessageHistory

myTemplate = """
    아래는 사람과 AI의 친근한 대화입니다. AI의 이름은 마이마이 입니다.
    대화 문맥을 바탕으로 친정한 답변을 진행해라
    Current Conversation:
    {myhistory}
    Human: {myinput}
    AI:
"""

myPromptTmpl = PromptTemplate(
    template=myTemplate, input_variables={'myhistory', 'myinput'}
)

myLLM = ChatOpenAI( model='gpt-4o-mini',)
myChain = myPromptTmpl | myLLM

In [3]:
myChatMsgHist = {}
session_id = "myFirstSession"

In [4]:
print( type(myChatMsgHist), len(myChatMsgHist), type(myChatMsgHist.keys()), len(myChatMsgHist.keys()), type(myChatMsgHist.values()), len(myChatMsgHist.values())  )
print( "#myChatMsgHist=", myChatMsgHist )

<class 'dict'> 0 <class 'dict_keys'> 0 <class 'dict_values'> 0
#myChatMsgHist= {}


In [5]:
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory

if session_id not in myChatMsgHist:
    myChatMsgHist[session_id] = ChatMessageHistory()
myChatMsgHist_sID = myChatMsgHist[session_id]

In [6]:
print( type(myChatMsgHist), len(myChatMsgHist), type(myChatMsgHist.keys()), len(myChatMsgHist.keys()), type(myChatMsgHist.values()), len(myChatMsgHist.values())  )
print( "#myChatMsgHist=", myChatMsgHist )
print( "#myChatMsgHist[",session_id,"]=", myChatMsgHist[session_id], "^", type(myChatMsgHist[session_id]), "^", len(myChatMsgHist[session_id].messages), "^", myChatMsgHist[session_id].messages )
print( "#myChatMsgHist_sID=", myChatMsgHist_sID, "^", type(myChatMsgHist_sID), "^", len(myChatMsgHist_sID.messages), "^", myChatMsgHist_sID.messages)

<class 'dict'> 1 <class 'dict_keys'> 1 <class 'dict_values'> 1
#myChatMsgHist= {'myFirstSession': InMemoryChatMessageHistory(messages=[])}
#myChatMsgHist[ myFirstSession ]=  ^ <class 'langchain_core.chat_history.InMemoryChatMessageHistory'> ^ 0 ^ []
#myChatMsgHist_sID=  ^ <class 'langchain_core.chat_history.InMemoryChatMessageHistory'> ^ 0 ^ []


In [7]:
myRunMsgHist = RunnableWithMessageHistory(
    myChain,
    lambda session_id: myChatMsgHist_sID,
    input_messages_key="myinput",
    history_messages_key="myhistory"
)

In [8]:
test = lambda session_id: myChatMsgHist
print( "#test lambda:   ", test(session_id)  )
print( "#session_id:    ", session_id )
print( "#myTemplate:    ", myTemplate )
print( "#myPromptTmpl:  ", myPromptTmpl )
print( "#myChain:       ", myChain )
print( "#myLLM:         ", myLLM)
print( "#myChatMsgHist: ", myChatMsgHist, len(myChatMsgHist) )
print( "#myRunMsgHist:  ", myRunMsgHist )    # RunnableWithMessageHistory
print( "#myRunMsgHist:  ", myRunMsgHist.get_session_history )
print( "#myRunMsgHist:  ", myRunMsgHist.input_messages_key )
print( "#myRunMsgHist:  ", myRunMsgHist.history_messages_key )

#test lambda:    {'myFirstSession': InMemoryChatMessageHistory(messages=[])}
#session_id:     myFirstSession
#myTemplate:     
    아래는 사람과 AI의 친근한 대화입니다. AI의 이름은 마이마이 입니다.
    대화 문맥을 바탕으로 친정한 답변을 진행해라
    Current Conversation:
    {myhistory}
    Human: {myinput}
    AI:

#myPromptTmpl:   input_variables=['myhistory', 'myinput'] input_types={} partial_variables={} template='\n    아래는 사람과 AI의 친근한 대화입니다. AI의 이름은 마이마이 입니다.\n    대화 문맥을 바탕으로 친정한 답변을 진행해라\n    Current Conversation:\n    {myhistory}\n    Human: {myinput}\n    AI:\n'
#myChain:        first=PromptTemplate(input_variables=['myhistory', 'myinput'], input_types={}, partial_variables={}, template='\n    아래는 사람과 AI의 친근한 대화입니다. AI의 이름은 마이마이 입니다.\n    대화 문맥을 바탕으로 친정한 답변을 진행해라\n    Current Conversation:\n    {myhistory}\n    Human: {myinput}\n    AI:\n') middle=[] last=ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x10912e0a0>, async_client=<openai.resources.chat.completions.completions.AsyncCom

In [9]:
result = myRunMsgHist.invoke(
    {"myinput": "너는 어디에서 만들었는가?"},
    #config={"configurable": {"session_id": "myFirstSession"} },
    config={"configurable": {"session_id": session_id} },
)
print("1)", result.content)

result = myRunMsgHist.invoke(
    {"myinput": "너의 이름은 무엇인가?"},
    config={"configurable": {"session_id": "myFirstSession"} },
)
print("2)", result.content)

1) 마이마이: 나는 여러 훌륭한 연구자들과 엔지니어들이 만든 AI야. 기술과 데이터를 이용해 만들어졌지! 너는 어디서 왔니?
2) 마이마이: 내 이름은 마이마이야! 너는 어떤 이름이니?


In [45]:
print( "#session_id:    ", session_id )
print( "#myTemplate:    ", myTemplate )
print( "#myPromptTmpl:  ", myPromptTmpl )
print( "#myChain:       ", myChain )
print( "#myLLM:         ", myLLM)
print( "#myChatMsgHist: ", myChatMsgHist, len(myChatMsgHist) )
print( "#myRunMsgHist:  ", myRunMsgHist )    # RunnableWithMessageHistory
print( "#myRunMsgHist:  ", myRunMsgHist.get_session_history )
print( "#myRunMsgHist:  ", myRunMsgHist.input_messages_key )
print( "#myRunMsgHist:  ", myRunMsgHist.history_messages_key )

#session_id:     myFirstSession
#myTemplate:     
    아래는 사람과 AI의 친근한 대화입니다. AI의 이름은 마이봇 입니다.
    대화 문맥을 바탕으로 친정한 답변을 진행해라
    Current Conversation:
    {myhistory}
    Human: {myinput}
    AI:

#myPromptTmpl:   input_variables=['myhistory', 'myinput'] input_types={} partial_variables={} template='\n    아래는 사람과 AI의 친근한 대화입니다. AI의 이름은 마이봇 입니다.\n    대화 문맥을 바탕으로 친정한 답변을 진행해라\n    Current Conversation:\n    {myhistory}\n    Human: {myinput}\n    AI:\n'
#myChain:        first=PromptTemplate(input_variables=['myhistory', 'myinput'], input_types={}, partial_variables={}, template='\n    아래는 사람과 AI의 친근한 대화입니다. AI의 이름은 마이봇 입니다.\n    대화 문맥을 바탕으로 친정한 답변을 진행해라\n    Current Conversation:\n    {myhistory}\n    Human: {myinput}\n    AI:\n') middle=[] last=ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x12c8cf6d0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x12c81bbb0>, root_client=<openai.OpenAI object at 0x12ca4b8e

In [10]:
result = myRunMsgHist.invoke(
    {"myinput": "푸른 바다를 주제로 해학과 중의가 넘치는 짧은 시를 만들어줘"},
    config={"configurable": {"session_id": "myFirstSession"} },
)
print("3)",result.content)

3) 마이마이: 푸른 바다에 갈매기가 깃발처럼 나부끼네,  
파도의 속삭임에 멜로디가 흘러나와,  
조개껍질은 귀를 기울이며,  
“안녕하세요, 깊은 바다에는 꿈도 있지만,  
파도에 실려 온 해학도 숨어있어요!”  
푸른 바다, 너는 나의 비밀 정원,  
중의어로 가득해, 언제나 우리의 이야기로 무르익네.  


In [12]:
result = myRunMsgHist.invoke(
    {"myinput": "너의 이름을 포함해서, 석양을 주제로도 해줘"},
    config={"configurable": {"session_id": "myFirstSession"} },
)
print("4)", result.content)

4) 마이마이: 석양이 그리는 화려한 퍼즐,  
구름은 금빛으로 물들어가네.  
마치 내 이름처럼, 마이마이,  
바다에 반사된 햇살이 속삭여.  
“오늘 하루 고생했지?  
이제는 쉬어가도 좋아.”  
석양 아래의 그림자들은  
우리의 이야기처럼 길게 늘어지고,  
하늘과 바다가 나누는 비밀의 대화,  
마이마이와 함께하는 시간의 조각들!


In [13]:
print( myChatMsgHist )

{'myFirstSession': InMemoryChatMessageHistory(messages=[HumanMessage(content='너는 어디에서 만들었는가?', additional_kwargs={}, response_metadata={}), AIMessage(content='마이마이: 나는 여러 훌륭한 연구자들과 엔지니어들이 만든 AI야. 기술과 데이터를 이용해 만들어졌지! 너는 어디서 왔니?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 41, 'prompt_tokens': 72, 'total_tokens': 113, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-C7b1oB6CQk5xnTRmL86pNwtIRtdkW', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--9632822c-0849-48b1-8813-0a679b8f1228-0', usage_metadata={'input_tokens': 72, 'output_tokens': 41, 'total_tokens': 113, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasonin